# **Activity Agent**

The activity agent will be responsible for predicting the user's activity for a given day. It will produce activity probabilities for the given user and the given day on an hourly level. These hourly activity probabilities will be processed by the subsequent recommendation agent to determine for which hours our framework should make recommendations. As input the activity agent receives the prepared data from the preparation agent.

We will define an activity agent class, add necessary functions to the agent, explain each step and build our final activity agent pipeline. A complete activity agent class definition will be available in the appendix to this notebook.

## **1. Preparing the Environment**

### **1.1 Loading the Libraries and the Data**

Steps to set up the environment for the activity agent include loading the necessary libraries, data and preceding agents.


In [1]:
import pandas as pd
import numpy as np

from helper_functions import Helper
from agents import Preparation_Agent
from InputError import InputError

helper = Helper()

In [2]:
# ML Models
import xgboost as xgb
import xgboost
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression


In [4]:
DATA_PATH = '../data/'

In [5]:
# load household data
household = helper.load_household(DATA_PATH, 1)

### **1.2 Calling the Preparation Agent**

To receive the necessary input for the activity agent, we will call the activity pipeline function of the preparation agent. We will specify the input data and the necessary configurations. As a quick validation that the preparation pipeline worked properly, we will plot the hourly activity of the household.

The current input data includes only a few time features and activity lags. However, our agent architecture allows to expand the inputs to the activity agent to further features or further data sources (e.g. whether data). For the purpose of demonstrating the functionality of our framework, we only use the inputs from the prepared REFIT data. 


In [6]:
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_params = {
    'resample_param': '60T'
}

activity_params = {
    'active_appliances': ['Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site'],
    'threshold': .15
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

In [7]:
# calling the preparation pipeline
prep = Preparation_Agent(household)
df = prep.pipeline_activity(household, activity_pipe_params)
df

[outlier truncation: Unix]: 100%|██████████| 6960008/6960008 [00:10<00:00, 692902.03it/s]
[outlier truncation: Unix]: 0 outliers were truncated.


[outlier truncation: Aggregate]: 100%|██████████| 6959964/6959964 [00:09<00:00, 719357.64it/s]
[outlier truncation: Aggregate]: 853913 outliers were truncated.


[outlier truncation: Fridge]: 100%|██████████| 1611328/1611328 [00:03<00:00, 494595.84it/s]
[outlier truncation: Fridge]: 60120 outliers were truncated.


[outlier truncation: Chest Freezer]: 100%|██████████| 2395354/2395354 [00:04<00:00, 567576.17it/s]
[outlier truncation: Chest Freezer]: 207420 outliers were truncated.


[outlier truncation: Upright Freezer]: 100%|██████████| 2800342/2800342 [00:04<00:00, 580450.85it/s]
[outlier truncation: Upright Freezer]: 197818 outliers were truncated.


[outlier truncation: Tumble Dryer]: 100%|██████████| 28117/28117 [00:00<00:00, 141607.78it/s]
[outlier truncation: Tumble Dryer]: 5934 outliers were truncated.


[outlier truncation: Washing M

,activity,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,,
2013-10-09 13:00:00,0,13,NaN,NaN,NaN,0,0,0,0,0,1
2013-10-09 14:00:00,0,14,NaN,NaN,NaN,0,0,0,0,0,1
2013-10-09 15:00:00,0,15,NaN,NaN,NaN,0,0,0,0,0,1
2013-10-09 16:00:00,0,16,NaN,NaN,NaN,0,0,0,0,0,1
2013-10-09 17:00:00,1,17,NaN,NaN,NaN,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2015-07-10 07:00:00,0,7,0.0,0.0,0.0,0,0,0,0,0,0
2015-07-10 08:00:00,0,8,0.0,0.0,0.0,0,0,0,0,0,0
2015-07-10 09:00:00,0,9,0.0,0.0,0.0,0,0,0,0,0,0


In [ ]:
import matplotlib.pyplot as plt

df.groupby(by='hour').mean()['activity'].plot();
plt.title('Average hourly activity')
plt.ylim(-0.1, 1.1);


## **2. Creating the Activity Agent**

In [8]:
class Activity_Agent:

    def __init__(self, activity_input_df):
        self.input = activity_input_df

### **2.1 Train Test Split**

As our recommendation framework will provide recommendations for a given day, our activity agent must be able to provide activity predictions for each day. However, for each day to be predicted we are only allowed to use information from the past to prevent leakage. Therefore, we will create a custom train test split function to meet the time series requirements. 

Our train test split function will enable us to conveniently split our data in the train and test data sets for each day to be predicted while allowing for further configurations of the split (i.e. size of the train and test data sets).

**X_test**

In [9]:
def get_Xtest(self, df, date, time_delta='all', target='activity'):
    import pandas as pd
    from helper_functions import Helper

    helper = Helper()
    
    if time_delta == 'all':
        output = df.loc[pd.to_datetime(date):, df.columns != target]
    else:
        df = helper.get_timespan(df, date, time_delta)
        output = df.loc[:, df.columns != target]
    return output

# add to Activity agent
setattr(Activity_Agent, 'get_Xtest', get_Xtest)
del get_Xtest

In [10]:
date = '2014-01-01'
activity = Activity_Agent(df)
X_test = activity.get_Xtest(df, date)
X_test

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2014-01-01 00:00:00,0,0.0,1.0,1.0,0,0,0,0,0,1
2014-01-01 01:00:00,1,1.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 02:00:00,2,1.0,0.0,1.0,0,0,0,0,0,1
2014-01-01 03:00:00,3,1.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 04:00:00,4,0.0,1.0,0.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2015-07-10 07:00:00,7,0.0,0.0,0.0,0,0,0,0,0,0
2015-07-10 08:00:00,8,0.0,0.0,0.0,0,0,0,0,0,0
2015-07-10 09:00:00,9,0.0,0.0,0.0,0,0,0,0,0,0


In [11]:
date = '2014-01-01'
activity = Activity_Agent(df)
time_delta = {'days': 1, 'seconds': -1}
X_test = activity.get_Xtest(df, date, time_delta=time_delta)
X_test

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2014-01-01 00:00:00,0,0.0,1.0,1.0,0,0,0,0,0,1
2014-01-01 01:00:00,1,1.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 02:00:00,2,1.0,0.0,1.0,0,0,0,0,0,1
2014-01-01 03:00:00,3,1.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 04:00:00,4,0.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 05:00:00,5,1.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 06:00:00,6,0.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 07:00:00,7,0.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 08:00:00,8,0.0,1.0,0.0,0,0,0,0,0,1


In [11]:
date = '2014-01-01'
activity = Activity_Agent(df)
time_delta = {'days': 7, 'seconds': -1}
X_test = activity.get_Xtest(df, date, time_delta=time_delta)
X_test

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2014-01-01 00:00:00,0,0.0,1.0,1.0,0,0,0,0,0,1
2014-01-01 01:00:00,1,1.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 02:00:00,2,1.0,0.0,1.0,0,0,0,0,0,1
2014-01-01 03:00:00,3,1.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 04:00:00,4,0.0,1.0,0.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2014-01-07 19:00:00,19,1.0,1.0,1.0,0,0,0,0,1,0
2014-01-07 20:00:00,20,1.0,1.0,1.0,0,0,0,0,1,0
2014-01-07 21:00:00,21,1.0,1.0,1.0,0,0,0,0,1,0


**y_test**

In [12]:
def get_ytest(self, df, date, time_delta='all', target='activity'):
    import pandas as pd
    from helper_functions import Helper

    helper = Helper()
    
    if time_delta == 'all':
        output = df.loc[pd.to_datetime(date):, target]
    else:
        output = helper.get_timespan(df, date, time_delta)[target]
    return output

# add to Activity agent
setattr(Activity_Agent, 'get_ytest', get_ytest)
del get_ytest

In [11]:
date = '2014-01-01'
activity = Activity_Agent(df)
y_test = activity.get_ytest(df, date)
y_test

Time
2014-01-01 00:00:00    1
2014-01-01 01:00:00    0
2014-01-01 02:00:00    1
2014-01-01 03:00:00    1
2014-01-01 04:00:00    1
                      ..
2015-07-10 07:00:00    0
2015-07-10 08:00:00    0
2015-07-10 09:00:00    0
2015-07-10 10:00:00    0
2015-07-10 11:00:00    0
Freq: 60T, Name: activity, Length: 13332, dtype: int64

In [13]:
date = '2014-01-01'

activity = Activity_Agent(df)
time_delta = {'days': 7, 'seconds': -1}

y_test = activity.get_ytest(df, date, time_delta=time_delta)
y_test

Time
2014-01-01 00:00:00    1
2014-01-01 01:00:00    0
2014-01-01 02:00:00    1
2014-01-01 03:00:00    1
2014-01-01 04:00:00    1
                      ..
2014-01-07 19:00:00    0
2014-01-07 20:00:00    1
2014-01-07 21:00:00    1
2014-01-07 22:00:00    1
2014-01-07 23:00:00    1
Freq: 60T, Name: activity, Length: 168, dtype: int64

**X_train**

In [14]:
# start = int (e.g. -100) --> 100 days before the day to be predicted will represent the train data
def get_Xtrain(self, df, date, start='2013-11-01', target='activity'):
    import pandas as pd

    if type(start) == int:
        start = pd.to_datetime(date) + pd.Timedelta(days= start)
        start = pd.to_datetime('2013-11-01') if start < pd.to_datetime('2013-11-01') else start
    else:
        start = pd.to_datetime(start)
    end = pd.to_datetime(date) + pd.Timedelta(seconds=-1)
    return df.loc[start:end, df.columns != target]

# add to Activity agent
setattr(Activity_Agent, 'get_Xtrain', get_Xtrain)
del get_Xtrain

In [16]:
date = '2014-01-01'

activity = Activity_Agent(df)
X_train = activity.get_Xtrain(df, date)
X_train

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2013-11-01 00:00:00,0,1.0,0.0,0.0,0,0,0,0,0,0
2013-11-01 01:00:00,1,1.0,1.0,0.0,0,0,0,0,0,0
2013-11-01 02:00:00,2,0.0,0.0,1.0,0,0,0,0,0,0
2013-11-01 03:00:00,3,0.0,0.0,1.0,0,0,0,0,0,0
2013-11-01 04:00:00,4,1.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,19,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 20:00:00,20,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 21:00:00,21,1.0,1.0,0.0,0,0,0,0,1,0


In [15]:
date = '2014-01-01'

activity = Activity_Agent(df)
X_train = activity.get_Xtrain(df, date, start=-1)
X_train

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2013-12-31 00:00:00,0,1.0,1.0,0.0,0,0,0,0,1,0
2013-12-31 01:00:00,1,1.0,0.0,1.0,0,0,0,0,1,0
2013-12-31 02:00:00,2,0.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 03:00:00,3,0.0,0.0,0.0,0,0,0,0,1,0
2013-12-31 04:00:00,4,1.0,0.0,1.0,0,0,0,0,1,0
2013-12-31 05:00:00,5,0.0,0.0,0.0,0,0,0,0,1,0
2013-12-31 06:00:00,6,0.0,0.0,0.0,0,0,0,0,1,0
2013-12-31 07:00:00,7,1.0,0.0,0.0,0,0,0,0,1,0
2013-12-31 08:00:00,8,1.0,0.0,0.0,0,0,0,0,1,0


In [18]:
date = '2014-01-01'

activity = Activity_Agent(df)
X_train = activity.get_Xtrain(df, date, start=-10)
X_train

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2013-12-22 00:00:00,0,0.0,0.0,1.0,0,0,1,0,0,0
2013-12-22 01:00:00,1,0.0,0.0,1.0,0,0,1,0,0,0
2013-12-22 02:00:00,2,1.0,0.0,0.0,0,0,1,0,0,0
2013-12-22 03:00:00,3,1.0,0.0,1.0,0,0,1,0,0,0
2013-12-22 04:00:00,4,0.0,0.0,0.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,19,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 20:00:00,20,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 21:00:00,21,1.0,1.0,0.0,0,0,0,0,1,0


In [16]:
date = '2014-01-01'

activity = Activity_Agent(df)
X_train = activity.get_Xtrain(df, date, start=-9999)
X_train

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2013-11-01 00:00:00,0,1.0,0.0,0.0,0,0,0,0,0,0
2013-11-01 01:00:00,1,1.0,1.0,0.0,0,0,0,0,0,0
2013-11-01 02:00:00,2,0.0,0.0,1.0,0,0,0,0,0,0
2013-11-01 03:00:00,3,0.0,0.0,1.0,0,0,0,0,0,0
2013-11-01 04:00:00,4,1.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,19,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 20:00:00,20,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 21:00:00,21,1.0,1.0,0.0,0,0,0,0,1,0


**y_train**

In [17]:
def get_ytrain(self, df, date, start='2013-11-01', target='activity'):
    import pandas as pd

    if type(start) == int:
        start = pd.to_datetime(date) + pd.Timedelta(days= start)
        start = pd.to_datetime('2013-11-01') if start < pd.to_datetime('2013-11-01') else start
    else:
        start = pd.to_datetime(start)
    end = pd.to_datetime(date) + pd.Timedelta(seconds=-1)
    return df.loc[start:end, target]

# add to Activity agent
setattr(Activity_Agent, 'get_ytrain', get_ytrain)
del get_ytrain

In [20]:
date = '2014-01-01'
activity = Activity_Agent(df)
y_train = activity.get_ytrain(df, date)
y_train

Time
2013-11-01 00:00:00    0
2013-11-01 01:00:00    0
2013-11-01 02:00:00    0
2013-11-01 03:00:00    0
2013-11-01 04:00:00    0
                      ..
2013-12-31 19:00:00    1
2013-12-31 20:00:00    1
2013-12-31 21:00:00    1
2013-12-31 22:00:00    1
2013-12-31 23:00:00    1
Freq: 60T, Name: activity, Length: 1464, dtype: int64

In [18]:
date = '2014-01-01'
activity = Activity_Agent(df)
y_train = activity.get_ytrain(df, date, start=-1)
y_train

Time
2013-12-31 00:00:00    0
2013-12-31 01:00:00    1
2013-12-31 02:00:00    1
2013-12-31 03:00:00    1
2013-12-31 04:00:00    0
2013-12-31 05:00:00    1
2013-12-31 06:00:00    0
2013-12-31 07:00:00    0
2013-12-31 08:00:00    0
2013-12-31 09:00:00    0
2013-12-31 10:00:00    0
2013-12-31 11:00:00    0
2013-12-31 12:00:00    1
2013-12-31 13:00:00    1
2013-12-31 14:00:00    1
2013-12-31 15:00:00    1
2013-12-31 16:00:00    1
2013-12-31 17:00:00    1
2013-12-31 18:00:00    1
2013-12-31 19:00:00    1
2013-12-31 20:00:00    1
2013-12-31 21:00:00    1
2013-12-31 22:00:00    1
2013-12-31 23:00:00    1
Freq: 60T, Name: activity, dtype: int64

In [22]:
date = '2014-01-01'
activity = Activity_Agent(df)
y_train = activity.get_ytrain(df, date, start=-10)
y_train

Time
2013-12-22 00:00:00    1
2013-12-22 01:00:00    0
2013-12-22 02:00:00    1
2013-12-22 03:00:00    0
2013-12-22 04:00:00    1
                      ..
2013-12-31 19:00:00    1
2013-12-31 20:00:00    1
2013-12-31 21:00:00    1
2013-12-31 22:00:00    1
2013-12-31 23:00:00    1
Freq: 60T, Name: activity, Length: 240, dtype: int64

In [24]:
date = '2014-01-01'
activity = Activity_Agent(df)
y_train = activity.get_ytrain(df, date, start=-9999)
y_train

Time
2013-11-01 00:00:00    0
2013-11-01 01:00:00    0
2013-11-01 02:00:00    0
2013-11-01 03:00:00    0
2013-11-01 04:00:00    0
                      ..
2013-12-31 19:00:00    1
2013-12-31 20:00:00    1
2013-12-31 21:00:00    1
2013-12-31 22:00:00    1
2013-12-31 23:00:00    1
Freq: 60T, Name: activity, Length: 1464, dtype: int64

**train_test_split**

In [19]:
def train_test_split(self, df, date, train_start='2013-11-01', test_delta='all', target='activity'):
    X_train = self.get_Xtrain(df, date, start=train_start, target=target)
    y_train = self.get_ytrain(df, date, start=train_start, target=target)
    X_test = self.get_Xtest(df, date, time_delta=test_delta, target=target)
    y_test = self.get_ytest(df, date, time_delta=test_delta, target=target)
    return X_train, y_train, X_test, y_test

# add to Activity agent
setattr(Activity_Agent, 'train_test_split', train_test_split)
del train_test_split    

In [20]:
date = '2014-01-01'
activity = Activity_Agent(df)
X_train, y_train, X_test, y_test = activity.train_test_split(df, date)
df.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15335, 11), (1464, 10), (1464,), (13332, 10), (13332,))

In [21]:
date = '2014-01-01'
activity = Activity_Agent(df)
X_train, y_train, X_test, y_test = activity.train_test_split(df, date, train_start=-10, test_delta={'days': 10, 'seconds':-1})
df.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15335, 11), (240, 10), (240,), (240, 10), (240,))

In [22]:
date = '2014-01-01'
activity = Activity_Agent(df)
X_train, y_train, X_test, y_test = activity.train_test_split(df, date, test_delta={'days': 1, 'seconds':-1})
df.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15335, 11), (1464, 10), (1464,), (24, 10), (24,))

In [45]:
X_train

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2013-11-01 00:00:00,0,1.0,0.0,0.0,0,0,0,0,0,0
2013-11-01 01:00:00,1,1.0,1.0,0.0,0,0,0,0,0,0
2013-11-01 02:00:00,2,0.0,0.0,1.0,0,0,0,0,0,0
2013-11-01 03:00:00,3,0.0,0.0,1.0,0,0,0,0,0,0
2013-11-01 04:00:00,4,1.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,19,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 20:00:00,20,1.0,1.0,1.0,0,0,0,0,1,0
2013-12-31 21:00:00,21,1.0,1.0,0.0,0,0,0,0,1,0


In [46]:
X_test

,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
Time,,,,,,,,,,
2014-01-01 00:00:00,0,0.0,1.0,1.0,0,0,0,0,0,1
2014-01-01 01:00:00,1,1.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 02:00:00,2,1.0,0.0,1.0,0,0,0,0,0,1
2014-01-01 03:00:00,3,1.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 04:00:00,4,0.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 05:00:00,5,1.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 06:00:00,6,0.0,0.0,0.0,0,0,0,0,0,1
2014-01-01 07:00:00,7,0.0,1.0,0.0,0,0,0,0,0,1
2014-01-01 08:00:00,8,0.0,1.0,0.0,0,0,0,0,0,1


In [ ]:
y_test

### **2.2 Building Models**

Within this section, we will demonstrate how different models for predicting the user activity could be added to the activity agent. For this notebook, we chose to implement a pretty simple logistic regression using the statsmodel api which already provides promising results. 

The functionality of the activity agent will include building models, fitting models and predicting the user’s activity.


In [29]:
def fit_Logit(self, X, y):
    return LogisticRegression(random_state=0).fit(X, y)


# Other ML Models 
# ---------------------------------------------------------------------------------------------

def fit_knn(self, X, y):
    return KNeighborsClassifier(3).fit(X,y)

def fit_random_forest(self, X,y):
    return RandomForestClassifier(max_depth=5, n_estimators=240, max_features=1).fit(X,y)

def fit_ADA(self,X,y):
    return  AdaBoostClassifier().fit(X,y)
def fit_XGB(self, X,y):
    return xgb.XGBClassifier().fit(X,y)

# add to Activity agent
setattr(Activity_Agent, 'fit_Logit', fit_Logit)
del fit_Logit 

setattr(Activity_Agent, 'fit_knn', fit_knn)
del fit_knn    

setattr(Activity_Agent, 'fit_random_forest', fit_random_forest)
del fit_random_forest  

setattr(Activity_Agent, 'fit_ADA', fit_ADA)
del fit_ADA

setattr(Activity_Agent, 'fit_XGB', fit_XGB)
del fit_XGB
  

In [30]:
def fit(self, X, y, model_type):
    model = None
    if model_type == "logit":
        model = self.fit_smLogit(X, y)
    elif model_type == "ada":
        model = self.fit_ADA(X,y)
    elif model_type == "knn":
        model = self.fit_knn(X,y)
    elif model_type == "random forest":
        model = self.fit_random_forest(X,y)
    elif model_type == "xgboost":
        model = self.fit_XGB(X,y)
    else:
        raise InputError("Unknown model type.")
    return model


# add to Activity agent
setattr(Activity_Agent, 'fit', fit)
del fit

In [61]:
def predict(self, model, X):
    import numpy as np
    import pandas

    index = X_test.index.values
    if type(X) == pandas.core.series.Series:
        X = pd.DataFrame(X).transpose() 
    else:
        X=np.squeeze(np.asarray(X))     

    if type(model) == sklearn.linear_model.LogisticRegression:
        y_hat = model.predict(X)
    elif type(model) == sklearn.neighbors._classification.KNeighborsClassifier:
        y_hat = model.predict(X)
    elif type(model) == sklearn.ensemble._forest.RandomForestClassifier:
        y_hat = model.predict(X)
    elif type(model) ==  sklearn.ensemble._weight_boosting.AdaBoostClassifier:
        y_hat = model.predict(X)
    elif type(model) == xgboost.sklearn.XGBClassifier:
        y_hat = model.predict(X)
    else:
        raise InputError("Unknown model type.")
    y_hat = pd.Series(y_hat, index=index, name='Time')
    return y_hat
# add to Activity agent
setattr(Activity_Agent, 'predict', predict)
del predict

Time
2014-01-01 00:00:00    0
2014-01-01 01:00:00    1
2014-01-01 02:00:00    0
2014-01-01 03:00:00    0
2014-01-01 04:00:00    0
2014-01-01 05:00:00    0
2014-01-01 06:00:00    0
2014-01-01 07:00:00    0
2014-01-01 08:00:00    0
2014-01-01 09:00:00    0
2014-01-01 10:00:00    0
2014-01-01 11:00:00    0
2014-01-01 12:00:00    1
2014-01-01 13:00:00    0
2014-01-01 14:00:00    1
2014-01-01 15:00:00    1
2014-01-01 16:00:00    0
2014-01-01 17:00:00    0
2014-01-01 18:00:00    1
2014-01-01 19:00:00    0
2014-01-01 20:00:00    0
2014-01-01 21:00:00    1
2014-01-01 22:00:00    1
2014-01-01 23:00:00    1
Freq: 60T, Name: Time, dtype: int64

In [71]:
index = X_test.index

In [62]:
# fit models
model = activity.fit( X_train, y_train, 'xgboost')


[17:28:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [63]:
# predict
y_hat = activity.predict(model, X_test)


In [64]:
activity = Activity_Agent(df)
y_hat = activity.predict(model, X_test)
y_hat

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1])

### **2.3 Model Evaluation**

A drawback to our approach is that we are not able to apply conventional model evaluation techniques to our model. We will train our model for each day to account for newly available information. Hence, we have different train and test sets for each day and for each day different performance metric based on the respective data sets. Therefore, we created our own evaluation function. 

Our evaluation function will build a model, fit the model and predict the target for each day for a given prediction period. For each day and fitted model it will calculate a performance metric on the train data. We chose the Area Under the Receiver Operating Characteristic Curve (AUC) as performance metric for our binary classification task. As in our case the test data is only the current date to be predicted (i.e. 24 data cases in the test data, one for each hour of the day), calculating the AUC on the test date immediately would lead to a high volatility in the test AUC per day to be predicted. Therefore, we calculate the test AUC over the activity probabilities of all days after all days have been predicted. To summarize the train AUC in one score, we apply an average over all calculated train AUC scores.

In [39]:
def auc(self, y_true, y_hat):
    import sklearn.metrics
    return sklearn.metrics.roc_auc_score(y_true, y_hat)

# add to Activity agent
setattr(Activity_Agent, 'auc', auc)
del auc

In [40]:
def evaluate(self, df, model_type, split_params, predict_start='2014-01-01', predict_end=-1):
    import pandas as pd
    import numpy as np
    from tqdm import tqdm

    dates = pd.DataFrame(df.index).set_index(df.index)['Time'].apply(lambda date: str(date)[:10]).drop_duplicates()

    predict_start = pd.to_datetime(predict_start)
    predict_end = pd.to_datetime(dates[predict_end]) if type(predict_end) == int else pd.to_datetime(predict_end)

    dates = dates.loc[predict_start:predict_end]
    
    y_true = []
    y_hat_train = {}
    y_hat_test = []
    auc_train_dict = {}
    auc_test = []

    for date in tqdm(dates):
        # train test split
        #train_test_split(self, df, date, train_start='2013-11-01', test_delta='all', target='activity')
        X_train, y_train, X_test, y_test = self.train_test_split(df, date, **split_params)

        # fit model
        model = self.fit(X_train, y_train, model_type)

        # predict
        y_hat_train.update({date: self.predict(model, X_train)})
        y_hat_test += list(self.predict(model, X_test))

        # evaluate train data
        auc_train_dict.update({date: self.auc(y_train, list(y_hat_train.values())[-1])})
        
        y_true += list(y_test)
    
    auc_test = self.auc(y_true, y_hat_test)
    auc_train = np.mean(list(auc_train_dict.values()))

    return auc_train, auc_test, auc_train_dict


# add to Activity agent
setattr(Activity_Agent, 'evaluate', evaluate)
del evaluate

In [ ]:
split_params = {'train_start': '2013-11-01', 'test_delta': {'days':1, 'seconds':-1}, 'target': 'activity'}
activity = Activity_Agent(df)
auc_train, auc_test, auc_train_dict = activity.evaluate(df, 'random forest', split_params) 

In [ ]:
# printing the scores
print(f'[model performance] train auc score: {auc_train}\n[model performance] test auc score: {auc_test}')

### **2.4 Pipeline Function**

The pipeline function of the activity agent will receive a date as input, create the appropriate train test split, fit the selected model to the data and return the activity predictions for the day to be predicted. The subsequent recommendation agent will process these activity probabilities to provide recommendations to the user.

In [ ]:
def pipeline(self, df, date, model_type, split_params):
    X_train, y_train, X_test, y_test = self.train_test_split(df, date, **split_params)

    # fit model
    model = self.fit(X_train, y_train, model_type)

    # predict
    return self.predict(model, X_test)

# add to Activity agent
setattr(Activity_Agent, 'pipeline', pipeline)
del pipeline

In [ ]:
activity = Activity_Agent(df)

date = '2014-06-01'
split_params = {'train_start': '2013-11-01', 'test_delta': {'days':1, 'seconds':-1}, 'target': 'activity'}

output = activity.pipeline(df, date, 'random forest', split_params)
output

In [ ]:
output = pd.DataFrame(output)

In [ ]:
output.plot();
plt.ylim(-.1, 1.1);

## **Appendix A1: Complete Activity Agent Class**

In [ ]:
class Activity_Agent:
    def __init__(self, activity_input_df):
        self.input = activity_input_df

    # train test split
    # -------------------------------------------------------------------------------------------
    def get_Xtest(self, df, date, time_delta="all", target="activity"):
        import pandas as pd
        from helper_functions import Helper

        helper = Helper()

        if time_delta == "all":
            output = df.loc[pd.to_datetime(date) :, df.columns != target]
        else:
            df = helper.get_timespan(df, date, time_delta)
            output = df.loc[:, df.columns != target]
        return output

    def get_ytest(self, df, date, time_delta="all", target="activity"):
        import pandas as pd
        from helper_functions import Helper

        helper = Helper()

        if time_delta == "all":
            output = df.loc[pd.to_datetime(date) :, target]
        else:
            output = helper.get_timespan(df, date, time_delta)[target]
        return output

    def get_Xtrain(self, df, date, start="2013-11-01", target="activity"):
        import pandas as pd

        if type(start) == int:
            start = pd.to_datetime(date) + pd.Timedelta(days=start)
            start = (
                pd.to_datetime("2013-11-01")
                if start < pd.to_datetime("2013-11-01")
                else start
            )
        else:
            start = pd.to_datetime(start)
        end = pd.to_datetime(date) + pd.Timedelta(seconds=-1)
        return df.loc[start:end, df.columns != target]

    def get_ytrain(self, df, date, start="2013-11-01", target="activity"):
        import pandas as pd

        if type(start) == int:
            start = pd.to_datetime(date) + pd.Timedelta(days=start)
            start = (
                pd.to_datetime("2013-11-01")
                if start < pd.to_datetime("2013-11-01")
                else start
            )
        else:
            start = pd.to_datetime(start)
        end = pd.to_datetime(date) + pd.Timedelta(seconds=-1)
        return df.loc[start:end, target]

    def train_test_split(
        self, df, date, train_start="2013-11-01", test_delta="all", target="activity"
    ):
        X_train = self.get_Xtrain(df, date, start=train_start, target=target)
        y_train = self.get_ytrain(df, date, start=train_start, target=target)
        X_test = self.get_Xtest(df, date, time_delta=test_delta, target=target)
        y_test = self.get_ytest(df, date, time_delta=test_delta, target=target)
        return X_train, y_train, X_test, y_test

# Logit
# ---------------------------------------------------------------------------------------------

    def fit_smLogit(self, X, y):
        import statsmodels.api as sm

        return sm.Logit(y, X).fit(disp=False)

    # Other ML Models 
    # ---------------------------------------------------------------------------------------------

    def fit_knn(self, X, y):
        return KNeighborsClassifier(3).fit(X,y)

    def fit_random_forest(self, X,y):
        return RandomForestClassifier(max_depth=5, n_estimators=240, max_features=1).fit(X,y)

    def fit_ADA(self,X,y):
        return  AdaBoostClassifier().fit(X,y)

    def fit_XGB(self,X,y):
        return xgb.XGBClassifier().fit(X,y)

    def fit(self, X, y, model_type):
        model = None
        if model_type == "logit":
            model = self.fit_smLogit(X, y)
        elif model_type == "ada":
            model = self.fit_ADA(X,y)
        elif model_type == "knn":
            model = self.fit_knn(X,y)
        elif model_type == "random forest":
            model = self.fit_random_forest(X,y)
        elif model_type == "xgboost":
            model = self.fit_XGB(X,y)
        else:
            raise InputError("Unknown model type.")
        return model

    def predict(self, model, X):
        import statsmodels
        import numpy as np
        import pandas

        if type(X) == pandas.core.series.Series:
            X = pd.DataFrame(X).transpose() 
        else:
            X=np.squeeze(np.asarray(X))     

        if type(model) == statsmodels.discrete.discrete_model.BinaryResultsWrapper:
            y_hat = model.predict(X)
        elif type(model) == sklearn.neighbors._classification.KNeighborsClassifier:
            y_hat = model.predict(X)
        elif type(model) == sklearn.ensemble._forest.RandomForestClassifier:
            y_hat = model.predict(X)
        elif type(model) ==  sklearn.ensemble._weight_boosting.AdaBoostClassifier:
            y_hat = model.predict(X)
        elif type(model) == xgboost.sklearn.XGBClassifier:
            y_hat = model.predict(X)
        else:
            raise InputError("Unknown model type.")
        return y_hat


    def auc(self, y_true, y_hat):
        import sklearn.metrics

        return sklearn.metrics.roc_auc_score(y_true, y_hat)

    def plot_model_performance(self, auc_train, auc_test, ylim="default"):
        import matplotlib.pyplot as plt

        plt.plot(list(auc_train.keys()), list(auc_train.values()))
        plt.plot(list(auc_train.keys()), list(auc_test.values()))
        plt.xticks(list(auc_train.keys()), " ")
        plt.ylim(ylim) if ylim != "default" else None

    def evaluate(
        self, df, model_type, split_params, predict_start="2014-01-01", predict_end=-1, return_errors=False
    ):
        import pandas as pd
        import numpy as np
        from tqdm import tqdm

        dates = (
            pd.DataFrame(df.index)
            .set_index(df.index)["Time"]
            .apply(lambda date: str(date)[:10])
            .drop_duplicates()
        )
        predict_start = pd.to_datetime(predict_start)
        predict_end = (
            pd.to_datetime(dates.iloc[predict_end])
            if type(predict_end) == int
            else pd.to_datetime(predict_end)
        )
        dates = dates.loc[predict_start:predict_end]
        y_true = []
        y_hat_train = {}
        y_hat_test = []
        auc_train_dict = {}
        auc_test = []

        for date in tqdm(dates):
            errors = {}
            try:
                X_train, y_train, X_test, y_test = self.train_test_split(
                    df, date, **split_params
                )

                # fit model
                model = self.fit(X_train, y_train, model_type)

                # predict
                y_hat_train.update({date: self.predict(model, X_train)})
                y_hat_test += list(self.predict(model, X_test))

                # evaluate train data
                auc_train_dict.update(
                    {date: self.auc(y_train, list(y_hat_train.values())[-1])}
                )
                y_true += list(y_test)
            except Exception as e:
                errors[date] = e

        auc_test = self.auc(y_true, y_hat_test)
        auc_train = np.mean(list(auc_train_dict.values()))

        if return_errors:
            return auc_train, auc_test, auc_train_dict, errors
        else:
            return auc_train, auc_test, auc_train_dict

    # pipeline function: predicting user activity
    # -------------------------------------------------------------------------------------------
    def pipeline(self, df, date, model_type, split_params):
        # train test split
        X_train, y_train, X_test, y_test = self.train_test_split(
            df, date, **split_params
        )

        # fit model
        model = self.fit(X_train, y_train, model_type)

        # predict
        return self.predict(model, X_test)